<a href="https://colab.research.google.com/github/csrajath/CSC_8980_NLP/blob/main/rcs1_TNLP_CSC_8980_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rajath Chikkatur Srinivasa
### 002552425

In [1]:
# importing libraries
import nltk, os
import pandas as pd
from collections import defaultdict
import random
from nltk.corpus import stopwords
from IPython.display import clear_output
from nltk.tokenize import word_tokenize  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# models
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
#sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# unzipping dataset
!tar -xvf /content/drive/MyDrive/Projects/NLP/review_polarity.tar.gz
clear_output()

1. Using NLTK tokenize all documents, separated by polarity, remove stop words, and list the top 20 most frequent tokens (and their counts) for the positive reviews, and the top 20 most frequent tokens (and their counts). What kind of things do you notice are different between the two sets? (30 points)

In [3]:
# collecting stopwords and punkts
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [4]:
pos_filepath = '/content/txt_sentoken/pos/'
neg_filepath = '/content/txt_sentoken/neg/'
pos_content = []
neg_content = []

In [5]:
for i in os.listdir(pos_filepath):
    file_content = open(os.path.join(pos_filepath,i), 'r').read()
    pos_content.append(file_content)
complete_pos_content = " ".join(pos_content)

In [6]:
for i in os.listdir(neg_filepath):
    file_content = open(os.path.join(neg_filepath,i), 'r').read()
    neg_content.append(file_content)
complete_neg_content = " ".join(neg_content)

In [7]:
#tokenizing positive
pos_tokens = word_tokenize(complete_pos_content) 
#tokenizing negative
neg_tokens = word_tokenize(complete_neg_content) 

In [8]:
#remove stopwords from positive tokens
rsw_pos_tokens = [word for word in pos_tokens if word.lower() not in stop_words]
#remove stopwords from negative tokens
rsw_neg_tokens = [word for word in neg_tokens if word.lower() not in stop_words]

In [9]:
p= defaultdict(int)
for i in rsw_pos_tokens:
    p[i]+=1
n= defaultdict(int)
for i in rsw_neg_tokens:
    n[i]+=1

In [10]:
pos_sort = sorted(p.items(), key=lambda x: x[1], reverse=True)[:20]
neg_sort = sorted(n.items(), key=lambda x: x[1], reverse=True)[:20]

In [11]:
print("top 20 most frequent tokens and their count for positive reviews are:")
for i in pos_sort:
  print('token:', i[0], 'count:',i[1])

top 20 most frequent tokens and their count for positive reviews are:
token: , count: 42448
token: . count: 33714
token: 's count: 9473
token: `` count: 8494
token: ) count: 6039
token: ( count: 6014
token: film count: 5186
token: one count: 2943
token: n't count: 2775
token: movie count: 2497
token: like count: 1713
token: ? count: 1570
token: : count: 1502
token: story count: 1231
token: also count: 1200
token: good count: 1190
token: even count: 1175
token: time count: 1171
token: would count: 1079
token: character count: 1067


In [12]:
print("top 20 most frequent tokens and their count for negative reviews are:")
for j in neg_sort:
  print('token:', j[0], 'count:',j[1])

top 20 most frequent tokens and their count for negative reviews are:
token: , count: 35269
token: . count: 32162
token: `` count: 9123
token: 's count: 8655
token: ) count: 5742
token: ( count: 5650
token: film count: 4257
token: n't count: 3442
token: movie count: 3174
token: one count: 2637
token: ? count: 2201
token: like count: 1832
token: : count: 1540
token: even count: 1381
token: would count: 1185
token: good count: 1126
token: time count: 1111
token: ! count: 1056
token: get count: 1039
token: bad count: 1019


* What kind of things do you notice are different between the two sets?
- Firstly the sentiment terms count and the term itself is different. This is obvious. For example: good v/s bad
- 

2. Using the code from previous lectures, build 3 polarity classifiers using the following parameters (20 points). Note: just train the models.\
a) For training: use 50% of the positive dataset and 70% of the negative dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.\
b) For training: use 50% of the negative dataset and 70% of the positive dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.\
c) For training: use 25% of the negative dataset and 25% of the positive dataset. For your model use: SVM with the TF-IDF vectorizer

a) For training: use 50% of the positive dataset and 70% of the negative dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

In [47]:
random.shuffle(pos_content)
random.shuffle(neg_content)
pos_m1 = len(pos_content[:int((50/100)*len(pos_content))])
neg_m1 = len(neg_content[:int((70/100)*len(pos_content))])
m1_pos_train = pos_content[:pos_m1]
m1_neg_train = neg_content[:neg_m1]
m1_pos_test = pos_content[pos_m1:]
m1_neg_test = neg_content[neg_m1:]

In [48]:
m1_train_data = m1_pos_train + m1_neg_train
m1_train_labels_1 = [1 for i in m1_pos_train]
m1_train_labels_0 = [0 for i in m1_neg_train]
m1_test_labels_1 = [1 for i in m1_pos_test]
m1_test_labels_0 = [0 for i in m1_neg_test]
m1_train_labels = m1_train_labels_1 + m1_train_labels_0 # 500-pos; 700-neg
m1_test_labels = m1_test_labels_1 + m1_test_labels_0 # 500-pos; 300-neg
#training NaiveBayes1
model_nb_1 = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb_1.fit(m1_train_data, m1_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

b) For training: use 50% of the negative dataset and 70% of the positive dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

In [49]:
random.shuffle(pos_content)
random.shuffle(neg_content)
pos_m2 = len(pos_content[:int((70/100)*len(pos_content))])
neg_m2 = len(neg_content[:int((50/100)*len(pos_content))])
m2_pos_train = pos_content[:pos_m2]
m2_neg_train = neg_content[:neg_m2]
m2_pos_test = pos_content[pos_m2:]
m2_neg_test = neg_content[neg_m2:]

In [50]:
#training NaiveBayes2
m2_train_data = m2_pos_train + m2_neg_train
m2_train_labels_1 = [1 for i in m2_pos_train]
m2_train_labels_0 = [0 for i in m2_neg_train]
m2_test_labels_1 = [1 for i in m2_pos_test]
m2_test_labels_0 = [0 for i in m2_neg_test]
m2_train_labels = m2_train_labels_1 + m2_train_labels_0
# m2_test_labels = m2_test_labels_1 + m2_test_labels_0
model_nb_2 = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb_2.fit(m2_train_data, m2_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

c) For training: use 25% of the negative dataset and 25% of the positive dataset. For your model use: SVM with the TF-IDF vectorizer

In [51]:
random.shuffle(pos_content)
random.shuffle(neg_content)
pos_m3 = len(pos_content[:int((25/100)*len(pos_content))])
neg_m3 = len(neg_content[:int((25/100)*len(pos_content))])
m3_pos_train = pos_content[:pos_m3]
m3_neg_train = neg_content[:neg_m3]
m3_pos_test = pos_content[pos_m3:]
m3_neg_test = neg_content[neg_m3:]

In [53]:
#training SVM
m3_train_data = m3_pos_train + m3_neg_train
m3_train_labels_1 = [1 for i in m3_pos_train]
m3_train_labels_0 = [0 for i in m3_neg_train]
m3_test_labels_1 = [1 for i in m3_pos_test]
m3_test_labels_0 = [0 for i in m3_neg_test]
m3_train_labels = m3_train_labels_1 + m3_train_labels_0
m3_test_labels = m3_test_labels_1 + m3_test_labels_0
model_svc_3 = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_svc_3.fit(m3_train_data, m3_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

3) Using the models from question 2, evaluate them on their individual rest of the dataset. This is, \
for a) 50% positive and 30% negative, \
for b) 50% negative and 30% positive, and \
for c) 75% negative and 75% positive. Calculate and show ONLY the following metrics for each model: Accuracy, Precision, Recall, Macro F1-score. (15 points)

In [54]:
# printing performance metrics for model-1
m1_name = 'NaiveBayes1-TfIdf'
m1_pred = model_nb_1.predict(m1_pos_test+m1_neg_test)
m1_accuracy = accuracy_score(m1_test_labels, m1_pred)
m1_precision = precision_score(m1_test_labels, m1_pred)
recall_model1 =  recall_score(m1_test_labels, m1_pred)
mf1_model_1 =  f1_score(m1_pred, m1_test_labels, average='macro')
m1_pvalues = [m1_name, m1_accuracy, m1_precision, recall_model1, mf1_model_1]
# print('Mean Absolute Error:', mean_absolute_error(test.target,labels))
# print('Mean Squared Error:', mean_squared_error(test.target, labels))  
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(test.target, labels)))

In [55]:
# printing performance metrics for model-2
m2_test_labels = m2_test_labels_1 + m2_test_labels_0
m2_name = 'NaiveBayes2-TfIdf'
m2_pred = model_nb_2.predict(m2_pos_test+m2_neg_test)
m2_accuracy = accuracy_score(m2_test_labels,m2_pred)
m2_precision = precision_score(m2_test_labels, m2_pred,m2_test_labels)
recall_model2 =  recall_score(m2_test_labels,m2_pred)
mf1_model_2 =  f1_score(m2_pred, m2_test_labels, average='macro')
m2_pvalues = [m2_name, m2_accuracy, m2_precision, recall_model2, mf1_model_2]
# print('Mean Absolute Error:', mean_absolute_error(test.target,labels))
# print('Mean Squared Error:', mean_squared_error(test.target, labels))  
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(test.target, labels)))

In [56]:
# printing performance metrics for model-3
m3_test_labels = m3_test_labels_1 + m3_test_labels_0
m3_name = 'Support Vector Machine'
m3_pred = model_svc_3.predict(m3_pos_test+m3_neg_test)
m3_accuracy = accuracy_score(m3_test_labels, m3_pred)
m3_precision = precision_score(m3_test_labels, m3_pred)
recall_model3 =  recall_score(m3_test_labels, m3_pred)
mf1_model_3 =  f1_score(m3_pred,m3_test_labels, average='macro')
m3_pvalues = [m3_name, m3_accuracy, m3_precision, recall_model3, mf1_model_3]
# print('Mean Absolute Error:', mean_absolute_error(test.target,labels))
# print('Mean Squared Error:', mean_squared_error(test.target, labels))  
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(test.target, labels)))

In [57]:
#creating a dataframe for metrics of all models performance metrixs
pm_list = [m3_pvalues, m2_pvalues, m1_pvalues]
pm_df = pd.DataFrame(pm_list, columns = ["model_name", "accuracy", "precison", "recall", 'macro_f1_score'])
pm_df

,model_name,accuracy,precison,recall,macro_f1_score
0,Support Vector Machine,0.797333,0.816761,0.766667,0.797143
1,NaiveBayes2-TfIdf,0.376250,0.375469,1.000000,0.274971
2,NaiveBayes1-TfIdf,0.378750,1.000000,0.006000,0.279437


4) Using the model performance metrics from question 3, answer the following questions. Please provide logical and intuitive rationale for your answers, simple answers like: because it has the best score, will not be sufficient. (40 points):\
a) What is the best performing model?\
b) Why do you think this is the best performing model?\
c) How does class imbalance play in determining polarity?\
d) Do you think either more data or a better model is a better approach for this
kind of task?

5) Using NLTK and VADER, calculate the sentiment score for all documents in the
positive polarity. Calculate the polarity threshold needed (and reasonable) to have the majority of the document labels match. Do the same for the negative class. Provide the threshold needed, the reason why you think this threshold is reasonable, and the accuracy percentage (how many documents are correctly labeled using this threshold). (45 points):

In [66]:
sia = SIA()

In [67]:
def get_scores(content, filename):
    sia_scores = sia.polarity_scores(content)
    
    return pd.Series({
        'filename': filename,
        'compound': sia_scores['compound'],
        'positive': sia_scores['pos'],
        'neutral': sia_scores['neu'],
        'negative': sia_scores['neg']
    })

In [68]:
df_pos_scores = pd.DataFrame([])

for i in os.listdir(pos_filepath):
    file_content = open(os.path.join(pos_filepath,i), 'r').read()
    df_1 = get_scores(file_content, i).to_frame().transpose()
    for index, row in df_1.iterrows():
      df_pos_scores = df_pos_scores.append({'file_name': row['filename'], 'compound': row['compound'],'positive': row['positive'], 'neutral': row['neutral'], 'negative': row['negative']}, ignore_index = True)

In [52]:
print('The sentiment score for all documents in the positive polarity is saved in the below dataframe')
df_pos_scores

The sentiment score for all documents in the positive polarity is saved in the below dataframe


,compound,file_name,negative,neutral,positive
0,0.9931,cv815_22456.txt,0.104,0.736,0.160
1,0.9996,cv274_25253.txt,0.083,0.713,0.204
2,0.9210,cv927_10681.txt,0.069,0.852,0.079
3,-0.9993,cv442_13846.txt,0.199,0.684,0.117
4,0.9973,cv551_10565.txt,0.087,0.753,0.160
...,...,...,...,...,...
995,-0.9568,cv117_24295.txt,0.169,0.674,0.157
996,-0.9861,cv602_8300.txt,0.100,0.848,0.052
997,0.9960,cv560_17175.txt,0.107,0.759,0.134
998,-0.9181,cv891_6385.txt,0.120,0.771,0.109


In [70]:
df_neg_scores = pd.DataFrame([])

for i in os.listdir(neg_filepath):
    file_content = open(os.path.join(neg_filepath,i), 'r').read()
    df_2 = get_scores(file_content, i).to_frame().transpose()
    for index, row in df_2.iterrows():
      df_neg_scores = df_neg_scores.append({'file_name': row['filename'], 'compound': row['compound'],'positive': row['positive'], 'neutral': row['neutral'], 'negative': row['negative']}, ignore_index = True)

In [71]:
print('The sentiment score for all documents in the negative polarity is saved in the below dataframe')
df_neg_scores

The sentiment score for all documents in the negative polarity is saved in the below dataframe


,compound,file_name,negative,neutral,positive
0,-0.8243,cv561_9484.txt,0.109,0.798,0.093
1,0.9932,cv135_12506.txt,0.083,0.755,0.161
2,-0.5106,cv568_17065.txt,0.116,0.772,0.112
3,-0.6371,cv297_10104.txt,0.134,0.732,0.135
4,0.9387,cv523_18285.txt,0.067,0.833,0.100
...,...,...,...,...,...
995,-0.9766,cv323_29633.txt,0.146,0.729,0.125
996,0.9981,cv574_23191.txt,0.096,0.735,0.169
997,0.8535,cv385_29621.txt,0.108,0.758,0.134
998,0.9793,cv936_17473.txt,0.113,0.731,0.156


Bonus (40 points): Repeat questions 2,3 and 4 removing all stopwords. Answer the
following questions: Did this change the results in any way? Why do you think so?

In [32]:
# for x,y in zip(filepath, file_content):
#   file_content = sia(y)
#   df = df.append()
